In [2]:
import os
from dotenv import load_dotenv
from openai import OpenAI
from langchain_core.output_parsers import JsonOutputParser
from langchain_openai import ChatOpenAI
from langchain_core import exceptions
from langchain.globals import set_llm_cache
from langchain.cache import SQLiteCache
from langchain_core.prompts import ChatPromptTemplate
from langsmith import traceable

from logging import getLogger

logger = getLogger(__name__)

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

set_llm_cache(SQLiteCache(database_path=".langchain.db"))

client = OpenAI(api_key=OPENAI_API_KEY, max_retries=5, timeout=60)

llm = ChatOpenAI(
    seed=311, temperature=1.0, model="gpt-4o-mini", openai_api_key=OPENAI_API_KEY, model_kwargs= {"response_format": {"type": "json_object"}})

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
                당신은 여행지를 추천해주는 ai입니다. 사용자가 지역을 얘기하면 지역에 맞는 여행지를 추천하세요.
                답변은 json형식으로 답변하시오

                예시) {{
                    "지역": [ "추천지1", "추천지2" ]
                }}
            
            """,
        ),
        ("human", "지역: {loc}"),
    ]
)


chunk_chain = prompt | llm | JsonOutputParser()



@traceable
async def excute(loc):
    try:
        result = chunk_chain.invoke({"loc": loc})

        return result

    except exceptions.LangChainException as e:
        logger.error(f"LangChain Error: {str(e)}")
        raise e





In [3]:
import asyncio



tasks = []
locs = ["서울", "부산", "대전"]
for loc in locs:
    tasks.append(excute(loc))

travel_loc = await asyncio.gather(*tasks)
travel_loc

[{'서울': ['경복궁', '남산타워', '홍대', '이태원', '북촌 한옥마을']},
 {'부산': ['해운대 해수욕장', '광안리 해수욕장', '부산 타워', '자갈치 시장', '범어사']},
 {'대전': ['한밭수목원', '대전오월드', '유림공원', '갑천', '대전 국립과학관']}]